In [1]:
import cv2 as cv
import numpy as np
import keyboard
import time

import kalman_filter
import Controlling_thymio
import global_planning
import vision
import loc_avoid
from ekf import ExtendedKalmanFilter
from ekf import apply_kalman

In [2]:
# Global constants here

IMAGE_WIDTH = 640 # width of the video feed

IMAGE_HEIGHT = 480 # height of the video feed

CAMERA_FPS = 30 # fps of the video feed

KEYBOARD_INPUT = "enter" # keyboard input to start

CAMERA_REFRESH_TIME = 30 # number of frame thrown away to allow the camera to focus in the meantime

SKIPPED_FRAME = 3 # get one frame every X

ACCEPTABLE_MOVEMENT = 100

In [3]:
from tdmclient import ClientAsync, aw
client = ClientAsync()
import Controlling_thymio

node = await client.wait_for_node()

await node.lock()
aw(node.lock())

await node.wait_for_variables({"acc"})

In [4]:
def check_angle(angle):
    if  isinstance(angle, np.ndarray):
        return float(angle[0])
    else:
        return float(angle)


In [ ]:
# 0 is laptop webcam, 1 is USB camera (!COMPUTER DEPENDENT!)
cap = cv.VideoCapture(0)

init = False # boolean set to True on keyboard input to start initialisation
is_init = False # boolean set to True once initialisation has succeeded
black_frame_remaining = 0 # numbers of frames to throw away
being_kidnapped = False # boolean set to True while the robot is being kidnapped
error_linear = [] # array of all linear errors
error_angle = [] # array of all angular errors
initial_turn = True # boolean set to True while the first orientation is not done

# variable initialized at False irrelevant of wether it actually is
robot_not_found = False # boolean set to True if robot not found for one frame
camera_hidden = False # boolean set to True if robot not found for multiple frames

# create a robot instance
robot_instance = Controlling_thymio.Robot()

# if unable to connect to the camera
if not (cap.isOpened()):
    print("Could not open video device")
else:
    while(True):
        # get the input frame by frame (shape (480,640,3))
        ret, frame = cap.read() 
        cap.set(cv.CAP_PROP_FPS, CAMERA_FPS) 
        frame = cv.resize(frame, (IMAGE_WIDTH,IMAGE_HEIGHT), interpolation=cv.INTER_CUBIC) 

        # wait until keyboard input to initialize vision
        if keyboard.is_pressed(KEYBOARD_INPUT):
            init = True

        # wait until keyboard input to initialize vision
        if keyboard.is_pressed("h"):
            if camera_hidden:
                camera_hidden = False
            else:
                camera_hidden = True

        # reinitialize the robot after a kidnapping
        if being_kidnapped: 
            if not Controlling_thymio.kidnapping(node):
                
                # to give time to the kidnapper to get out of the frame
                time.sleep(3)
                
                # reinitialize the starting position of the robot
                fop = vision.get_fop(frame, original_coordinates, new_coordinates)
                try:
                    start_x, start_y, robot_angle, width = vision.get_robot_position(fop)
                except:
                    break

                # reinitialize the global planning
                converted_obstacles = [[tuple(arr) for arr in sublist] for sublist in obstacles]
                converted_obstacles = global_planning.obstacle_dictionnary(converted_obstacles)
                goal = (objective_x, objective_y)
                robot_instance.update_coordinates(start_x, start_y, robot_angle, width)
                obstacles_named = global_planning.naming_points(converted_obstacles, robot_instance, goal)
                adg_list = global_planning.creating_adjacency_dictionnary(converted_obstacles, robot_instance, goal)
                path = global_planning.finding_path(adg_list, obstacles_named)

                black_frame_remaining = 0
                camera_hidden = False
                initial_turn = True
                is_init = True
                being_kidnapped = False
        
        # initialize 
        if init == True:
            try:
                # vision: start, obstacles and goal coordinates
                original_coordinates, new_coordinates = vision.get_fop_coordinates(frame)
                fop = vision.get_fop(frame, original_coordinates, new_coordinates)
                #start_x, start_y, robot_angle, width = vision.get_robot_position(fop)

                start_x, start_y, robot_angle, width = vision.get_robot_position(fop)

                #print(start_x,start_y)
                obstacles = vision.get_obstacles(fop, width)
                objective_x, objective_y = vision.get_objective(fop)

                # global planning
                converted_obstacles = [[tuple(arr) for arr in sublist] for sublist in obstacles]
                converted_obstacles = global_planning.obstacle_dictionnary(converted_obstacles)
                goal = (objective_x,objective_y)
                robot_instance.update_coordinates(start_x, start_y, robot_angle,width)
                obstacles_named = global_planning.naming_points(converted_obstacles,robot_instance,goal)
                adg_list = global_planning.creating_adjacency_dictionnary(converted_obstacles,robot_instance,goal)
                path = global_planning.finding_path(adg_list,obstacles_named)
                
                # extended kalman filter
                #ekf = ExtendedKalmanFilter(([start_x,start_y,robot_angle,0,0]))
                #ekf.set_time_t(time.time())

                kalman = kalman_filter.KalmanFilter(np.array([[start_x], [start_y], [robot_angle]], dtype=object))

                left_wheel_speed = 0
                right_wheel_speed = 0

                black_frame_remaining = 0
                camera_hidden = False
                being_kidnapped = False
                is_init = True
            except Exception as e:
                print("Initialisation failed: " + str(e))

            init = False

        if is_init:  
        ################################## MOTION CONTROL HERE ################################## 
            
            # get out of the loop if the robot is being kidnapped
            if Controlling_thymio.kidnapping(node):
                being_kidnapped = True
                is_init = False
                continue

            fop = vision.get_fop(frame, original_coordinates, new_coordinates)

            if camera_hidden:
                rows, columns = frame.shape[:2]
                
                for i in range(0, columns-1):
                    for j in range(0, rows-1):
                        fop[i][j] = [0,0,0]

            try:
                center_x, center_y, robot_angle, width = vision.get_robot_position(fop)
                if abs((center_x + center_y) - (robot_instance.center_x + robot_instance.center_y)) > ACCEPTABLE_MOVEMENT:
                    raise
                print("vision: ", center_x, center_y, robot_angle)
                robot_instance.update_coordinates(center_x, center_y, robot_angle, width)
            except:
                center_x = 0
                center_y = 0
                robot_angle = float(0)

            
            
            # ekf update
            position = np.array([[center_x], [center_y], [check_angle(robot_angle)]], dtype=float)
            speed = [float(left_wheel_speed),float(right_wheel_speed)]

            """ try:
                x = apply_kalman(ekf, position, speed)
                new_position = x[0:3]  # Extract the filtered position
                print('o')
            except ValueError as e:
                print(f"Error during Kalman filter update: {e}")
                new_position = [center_x, center_y, robot_angle] """
            [speed_x, speed_y, speed_angular] = kalman_filter.convert_input(robot_instance)
            new_speed = np.array([[speed_x], [speed_y], [speed_angular]], dtype=float)
            kalman.predict(new_speed)
            if center_x == 0:
                 kalman.update(kalman.x)
            else:
                kalman.update(position)
            
            
            # camera was uncovered, now covered
            if (center_x == 0 and center_y == 0) or camera_hidden:
    
                print("Kalman: ", kalman.x)
                robot_instance.update_coordinates(int(kalman.x[0]), int(kalman.x[1]), kalman.x[2], width)
            

            start_point = np.array([obstacles_named[path[0]][0],obstacles_named[path[0]][1]])
            end_point = np.array([obstacles_named[path[1]][0],obstacles_named[path[1]][1]])
            
            # traget point after the actual end point
            try:
                next_point = np.array([obstacles_named[path[2]][0],obstacles_named[path[2]][1]])
            except:
                next_point = [0, 0]

            # proximity avoidance
            if loc_avoid.check_local_nav(node, robot_instance, aw):
                left_wheel_speed, right_wheel_speed = loc_avoid.local_avoidance(node, robot_instance, aw)
                initial_turn = True     
            else:
                left_wheel_speed, right_wheel_speed, error_linear, error_angle, initial_turn, path = Controlling_thymio.compute_wheel_speed(initial_turn, start_point, end_point, next_point, error_linear, error_angle, robot_instance, path, aw, node)   

            robot_instance.update_speed(left_wheel_speed, right_wheel_speed)
            print("speeds: ", left_wheel_speed, right_wheel_speed)

            # draw everything
            cv.circle(fop, (robot_instance.center_x,robot_instance.center_y), radius=10, color=(0, 255, 0), thickness=-1)
            cv.circle(fop, (objective_x, objective_y), radius=10, color=(255, 0, 0), thickness=-1)
            cv.circle(fop, (start_x, start_y), radius=10, color=(0, 0, 255), thickness=-1)
            for obstacle in obstacles:
                for vertice in obstacle:
                    cv.circle(fop, (vertice[0],vertice[1]), radius=10, color=(0, 255, 0), thickness=-1)
            for i in range(len(path[:-1])):
                cv.line(fop,(obstacles_named[path[i]][0],obstacles_named[path[i]][1]),(obstacles_named[path[i+1]][0],obstacles_named[path[i+1]][1]),(255,0,0),5)
    
        #########################################################################################
        else:
            # to output the raw feed
            fop = frame

        cv.imshow('Raw',frame)
        cv.imshow('Processed',fop)

        await client.sleep(0.001)
        
        # Close all windows and get out of the loop if ESC is pressed
        if cv.waitKey(1) == 27:
            cv.destroyAllWindows()
            break

vision:  41 309 [0.89605538]
init turn
speeds:  156 143
vision:  42 309 [1.74546853]
speeds:  161 138
vision:  42 308 [1.80188699]
speeds:  174 125
vision:  42 306 [1.38544838]
speeds:  194 105
vision:  42 304 [1.50837752]
speeds:  200 79
vision:  43 303 [1.39612413]
speeds:  200 74
vision:  43 302 [1.24904577]
speeds:  200 64
vision:  44 301 [1.19028995]
speeds:  200 58
vision:  44 300 [1.21202566]
speeds:  200 50
vision:  45 297 [1.16590454]
speeds:  200 50
vision:  46 296 [0.78539816]
speeds:  200 50
vision:  46 294 [0.]
speeds:  200 50
vision:  48 293 [0.24497866]
speeds:  200 50
vision:  49 292 [0.24497866]
speeds:  200 50
vision:  50 291 [0.90975316]
speeds:  200 50
vision:  51 291 [1.39612413]
speeds:  200 50
vision:  53 289 [1.38544838]
speeds:  200 50
Kalman:  [[array([52.25525191])]
 [array([291.00848329])]
 [array([1.35977775])]]
speeds:  200 50
vision:  54 288 [0.24497866]


C:\Users\zalex\AppData\Local\Temp\ipykernel_16280\2494008470.py:167: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  robot_instance.update_coordinates(int(kalman.x[0]), int(kalman.x[1]), kalman.x[2], width)


speeds:  200 50
vision:  56 286 [0.08314123]
speeds:  200 50
Kalman:  [[array([56.130407])]
 [array([287.31645065])]
 [array([0.38063037])]]
speeds:  200 50
vision:  59 285 [0.5404195]
speeds:  200 50
Kalman:  [[array([58.77677249])]
 [array([286.40852111])]
 [array([0.5454379])]]
speeds:  200 50
vision:  62 284 [-0.5404195]
speeds:  200 50
Kalman:  [[array([61.64169613])]
 [array([284.39625751])]
 [array([-0.05960647])]]
speeds:  200 50
vision:  65 284 [0.35877067]
speeds:  200 50
Kalman:  [[array([64.64459089])]
 [array([284.49911271])]
 [array([0.26501723])]]
speeds:  200 50
vision:  69 285 [0.28605144]
speeds:  200 50
vision:  70 284 [0.23109067]
speeds:  200 50
vision:  72 284 [0.23109067]
speeds:  200 50
vision:  74 285 [0.1746722]
speeds:  200 61
Kalman:  [[array([73.96142548])]
 [array([284.87901837])]
 [array([0.27203656])]]
speeds:  200 50
vision:  76 284 [0.16514868]
speeds:  200 62
vision:  78 285 [-0.23109067]
speeds:  200 92
vision:  79 285 [-0.21866895]
speeds:  200 98
v

IndexError: index 613 is out of bounds for axis 0 with size 613